In [1]:
import pandas as pd

In [3]:
cost = pd.read_csv('./data/healthcare-expenditure-vs-gdp.csv')
gdp = pd.read_csv('./data/immunization-coverage-against-diphtheria-tetanus-and-pertussis-dtp3-vs-gdp-per-capita.csv')
population = pd.read_csv('./data/share-of-population-in-extreme-poverty.csv')

In [11]:
cost.head()

,Entity,Code,Year,"Current health expenditure per capita, PPP (current international $)","GDP per capita, PPP (constant 2021 international $)",Population (historical),World regions according to OWID
0,Afghanistan,AFG,2002,87.396550,1774.3087,21378081.0,NaN
1,Afghanistan,AFG,2003,86.260340,1815.9282,22733007.0,NaN
2,Afghanistan,AFG,2004,93.952965,1776.9182,23560598.0,NaN
3,Afghanistan,AFG,2005,105.886760,1908.1147,24404520.0,NaN
4,Afghanistan,AFG,2006,118.306465,1929.7239,25424050.0,NaN


In [5]:
gdp.head()

,Entity,Code,Year,"Share of one-year-olds who have received three doses of combined diphtheria, tetanus toxoid and pertussis-containing vaccine (DTP3)","GDP per capita, PPP (constant 2021 international $)",Population (historical),World regions according to OWID
0,Afghanistan,AFG,1980,4.0,NaN,13169267.0,NaN
1,Afghanistan,AFG,1981,3.0,NaN,11937540.0,NaN
2,Afghanistan,AFG,1982,5.0,NaN,10991329.0,NaN
3,Afghanistan,AFG,1983,5.0,NaN,10917939.0,NaN
4,Afghanistan,AFG,1984,16.0,NaN,11190179.0,NaN


In [6]:
population.head()

,Entity,Code,Year,$2.15 a day - Share of population in poverty,990305-annotations
0,Albania,ALB,1996,0.534846,NaN
1,Albania,ALB,2002,1.092647,NaN
2,Albania,ALB,2005,0.591086,NaN
3,Albania,ALB,2008,0.199925,NaN
4,Albania,ALB,2012,0.620691,NaN


In [10]:
population.isnull().sum()

Entity                                             0
Code                                             518
Year                                               0
$2.15 a day - Share of population in poverty       0
990305-annotations                              2670
dtype: int64

In [12]:
cost.shape

(59499, 7)

# Query 1

In [25]:
year_2021 = cost[cost['Year'] == 2021]
year_2021_sorted = year_2021.sort_values(by="Current health expenditure per capita, PPP (current international $)", ascending=True)
year_2021_sorted.head()
gdp_2021 = gdp[gdp['Year'] == 2021]

In [26]:
gdp_2021.head()

,Entity,Code,Year,"Share of one-year-olds who have received three doses of combined diphtheria, tetanus toxoid and pertussis-containing vaccine (DTP3)","GDP per capita, PPP (constant 2021 international $)",Population (historical),World regions according to OWID
41,Afghanistan,AFG,2021,55.0,2144.1665,4.000036e+07,NaN
523,Africa,NaN,2021,NaN,NaN,1.413750e+09,NaN
597,Africa (UN),NaN,2021,NaN,NaN,1.413753e+09,NaN
641,African Region (WHO),NaN,2021,73.0,NaN,NaN,NaN
689,Albania,ALB,2021,98.0,16353.8090,2.849591e+06,NaN


In [27]:
year_2021_sorted.merge(gdp_2021, how="left", left_on="Entity", right_on="Entity")

,Entity,Code_x,Year_x,"Current health expenditure per capita, PPP (current international $)","GDP per capita, PPP (constant 2021 international $)_x",Population (historical)_x,World regions according to OWID_x,Code_y,Year_y,"Share of one-year-olds who have received three doses of combined diphtheria, tetanus toxoid and pertussis-containing vaccine (DTP3)","GDP per capita, PPP (constant 2021 international $)_y",Population (historical)_y,World regions according to OWID_y
0,South Sudan,SSD,2021,33.766468,NaN,10865735.0,NaN,SSD,2021,69.0,NaN,10865735.0,NaN
1,Democratic Republic of Congo,COD,2021,45.612370,1313.6182,99148879.0,NaN,COD,2021,65.0,1313.6182,99148879.0,NaN
2,Madagascar,MDG,2021,56.670320,1599.4127,29691033.0,NaN,MDG,2021,57.0,1599.4127,29691033.0,NaN
3,Zimbabwe,ZWE,2021,63.136967,3184.7854,15797165.0,NaN,ZWE,2021,88.0,3184.7854,15797165.0,NaN
4,Burundi,BDI,2021,70.498190,836.6656,12965439.0,NaN,BDI,2021,94.0,836.6656,12965439.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
264,United States Virgin Islands,VIR,2021,NaN,46893.5500,87184.0,NaN,VIR,2021,NaN,46893.5500,87184.0,NaN
265,Vatican,VAT,2021,NaN,NaN,515.0,NaN,VAT,2021,NaN,NaN,515.0,NaN
266,Wallis and Futuna,WLF,2021,NaN,NaN,11544.0,NaN,WLF,2021,NaN,NaN,11544.0,NaN
267,Western Sahara,ESH,2021,NaN,NaN,558577.0,NaN,ESH,2021,NaN,NaN,558577.0,NaN


# Query 2